SAM 2024

#TP10 LSM Store

date document: 25/04/2024

## BOUZOURINE Hichem

L'objectif est de comprendre, par la pratique, les principes du stockage couramment mis en oeuvre dans les systemes big data tels que Cassandra et HBase.

## Préparation

In [ ]:
import os
import shutil
import random
import numpy as np
import time2024
from sortedcontainers import SortedDict

os.makedirs("data", exist_ok=True)

In [ ]:
# !rm -f *.csv

# Classe LSM : Log Structured Merge Storage

La classe LSM définit les principales fonctions du stockage basé sur LSM.

In [ ]:
class LSM:
  D_MAXSIZE =10


  def __init__(self):
    self.D = SortedDict()
    self.first_image = 0
    self.last_image = 0
    self.MAX_IMAGE = 2

  def set_max_image(self, m):
    self.MAX_IMAGE= m


  def update(self, k, v):
      self.D[k] = v

      # vider D dans un fichier si D a atteint sa taille maxi
      if len(self.D) == self.D_MAXSIZE:
        self.flush_D()
        # continuer avec un nouveau D vide
        self.D = SortedDict()


  def select(self, k):
    if k in self.D:
      return self.D[k]

    # recherche parmi les images du niveau L0
    nb_image = self.last_image - self.first_image + 1
    for m in range(nb_image):
      with open(f"data/image{self.last_image - m}.csv", "r") as f:
        for line in f:
          line = line.split(',')
          k1,v1 = int(line[0]), int(line[1])
          if k1 == k:
              return v1

    # recherche dans L1
    with open(f"data/L1.csv", "r") as f:
      for line in f:
        line = line.split(',')
        k1, v1 = int(line[0]), int(line[1])
        if k1 == k:
            return v1
    # not found
    return "NULL"



  def flush_D(self):
    if self.first_image == 0:
      self.first_image = 1

    self.last_image +=1

    with open(f"data/image{self.last_image}.csv", "w") as f:
      for k, v in self.D.items():
          f.write(str(k) + "," + str(v) + '\n')
      print(f"saved {len(self.D)} items in image {self.last_image}")


  def merge_L0_to_L1(self):
    # deplace les données de la premiere image de L0 vers le niveau L1
    nb_image = self.last_image - self.first_image + 1

    if nb_image > self.MAX_IMAGE:
      if os.path.isfile("data/L1.csv") :
        L1_new = []
        L0 = iter(self.read_log(f'image{self.first_image}.csv'))
        k0, line0 = next(L0, (None, ""))

        #fusionner L1 et l'image
        for k1, line1 in self.read_log("L1.csv"):
          while k0 is not None and k0<=k1:
              L1_new.append(line0)
              k0, line0 = next(L0, (None, ""))
          L1_new.append(line1)
        print('nb of lines in L1:', len(L1_new))

        # créer un nouveau fichier L1 contenant le resultat de la fusion
        with open("data/L1_new.csv", 'w') as file_L1_new:
          file_L1_new.write(''.join(L1_new))
        shutil.move("data/L1_new.csv", "data/L1.csv")
      else:
        # L1 ne contient rien, il suffit de déplacer l'image vers L1
        print(f"move image{self.first_image}.csv to L1.csv")
        shutil.move(f'data/image{self.first_image}.csv', "data/L1.csv")
      self.first_image +=1


  def read_log(self, file):
    with open("data/" + file) as f:
      for line in f:
        key = line.split(',')[0]
        yield key, line

print("class LSM defined")

class LSM defined


#Exercice 1

Comprendre la classe LSM définie ci dessus

## Ecriture de données dans le LSM



On effectue un test d'écriture de plusieurs couples dans le LSM

In [ ]:
# on nettoie le dossier ou seront stockés les données
shutil.rmtree("data")
os.makedirs("data", exist_ok=True)


nb_update = 100

rng = np.random.default_rng(seed=1)
keys = rng.integers(0, 100, nb_update)
values = rng.integers(0, 1000, nb_update)

store = LSM()

store.set_max_image(2)   # remplacer 2 par 4 ?

t1 = time.time()
for k,v in zip(keys,values):
  store.update(k,v)
  store.merge_L0_to_L1() # essayer d'enlever cette ligne

print("duree", round( (time.time() - t1) * 1000, 0), "ms")


saved 10 items in image 1
saved 10 items in image 2
saved 10 items in image 3
move image1.csv to L1.csv
saved 10 items in image 4
nb of lines in L1: 20
saved 10 items in image 5
nb of lines in L1: 30
saved 10 items in image 6
nb of lines in L1: 39
saved 10 items in image 7
nb of lines in L1: 47
saved 10 items in image 8
nb of lines in L1: 56
saved 10 items in image 9
nb of lines in L1: 66
duree 17.0 ms


1) Que se passe-t-il si on ajoute
store.set_max_image(4) au début du test d'écriture ?

2) Que se pass-t-il si on enlève la ligne
store.merge_L0_to_L1() dans le test?

## Lecture

In [ ]:
# Pour vérifier que la lecture est correcte, on connait à l'avance la valeur la plus récente de chaque k qu'on est censé lire
test_D = {}
for k,v in zip(keys,values):
  test_D[k]=v


t1 = time.time()
for k,v in test_D.items():
    # print("select", k)
    v1 = store.select(k)
    if v1 != v:
      print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("duree", round( (time.time() - t1) * 1000, 0), "ms")


Erreur pour la clé 27 ! La valeur retrouvée (219) n'est pas celle attendue (919)
Erreur pour la clé 32 ! La valeur retrouvée (645) n'est pas celle attendue (464)
Erreur pour la clé 97 ! La valeur retrouvée (NULL) n'est pas celle attendue (470)
Erreur pour la clé 96 ! La valeur retrouvée (NULL) n'est pas celle attendue (885)
duree 14.0 ms


# Exercice 2 : Effet de la fréquence de lecture

Proposer un test de lecture qui mette en évidence le fait que les clés lues fréquemment (par exemple une lecture toutes les 5 lectures) sont accédées plus rapidement que les clés rares (une lecture toutes les 50 lectures).

# Exercice 3 : Amélioration du stockage du niveau L1

On voit que le niveau L1 contient un seul fichier `L1.csv` qui peut grandir indéfiniment.
Cela tend à ralentir la recherche d'une clé "peu fréquente" qui serait dans le niveau L1.

Proposer une classe LSM_v2 pour laquelle le niveau L1 est composé de plusieurs fichiers F1, F2, ... tels que
*   chaque fichier contient au maximium N lignes (N= nombre de lignes dans une image),
*   les clés sont globalement triées dans les fichiers : toutes les clés du fichier Fi sont inférieures à celles du fichiers Fj (avec i<j)



# Exercice 4 : fusion entre plusieurs images

On voit que les imagess de L0 sont fusionnées individuellement vers le niveau L1.
Proposer une solution pour fusionner k images de L0 vers L1.
Indication: tenir compte du fait que les images peuvent contenir des clés identiques.